In [81]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

url = "https://coinmarketcap.com/"
driver.get(url)
time.sleep(10)
page_source = driver.page_source

if page_source:
    print("Page source successfully extracted!")
else:
    print("Failed to extract page source.")
driver.quit()

Page source successfully extracted!


In [82]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_argument('--headless') 

driver = webdriver.Chrome(options=chrome_options)

# URL of CoinMarketCap's page
url = "https://coinmarketcap.com/"
x=["","?page="]
crypto_data = []
for i in range(1,11):
    if i==0:
        url1=url+x[0]
    else :
        url1=url+x[1]+str(i)
    driver.get(url1)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sc-936354b2-3")))

    # page source
    page_source = driver.page_source

    #  parse the HTML
    soup = BeautifulSoup(page_source, 'html.parser')

    # cryptocurrency information
    crypto_elements = soup.find_all('a', class_='cmc-link')

    # names and links


    for element in crypto_elements:
        name_element = element.find('span')
        link_element = element.get('href')
        

        if name_element and link_element:
            crypto_data.append({
            'name': name_element.text.strip(),
            'link': link_element
            })

for index, data in enumerate(crypto_data):
    print(f"{index + 1}. Name: {data['name']}, Link: {data['link']}")

driver.quit()

1. Name: , Link: /currencies/toncoin/
2. Name: , Link: /currencies/shiba-inu/
3. Name: , Link: /currencies/stellar/
4. Name: , Link: /currencies/polkadot-new/
5. Name: , Link: /currencies/hedera/
6. Name: , Link: /currencies/bitcoin-cash/
7. Name: , Link: /currencies/uniswap/
8. Name: , Link: /currencies/pepe/
9. Name: , Link: /currencies/litecoin/
10. Name: , Link: /currencies/unus-sed-leo/
11. Name: , Link: /currencies/hyperliquid/
12. Name: , Link: /currencies/bitget-token-new/
13. Name: , Link: /currencies/near-protocol/
14. Name: , Link: /currencies/ethena-usde/
15. Name: , Link: /currencies/internet-computer/
16. Name: , Link: /currencies/aptos/
17. Name: , Link: /currencies/multi-collateral-dai/
18. Name: , Link: /currencies/aave/
19. Name: , Link: /currencies/mantle/
20. Name: , Link: /currencies/polygon-ecosystem-token/
21. Name: , Link: /currencies/cronos/
22. Name: , Link: /currencies/ethereum-classic/
23. Name: , Link: /currencies/virtual-protocol/
24. Name: , Link: /curren

In [83]:
import pandas as pd 
crypto_df = pd.DataFrame(crypto_data)
#data cleaning 
print(crypto_df.shape)

# Remove the general market sentiment
for index,row in crypto_df.iterrows():
    if row['link']=='/charts/#fear-and-greed-index':
        crypto_df.drop(index, inplace=True)
    
for index,row in crypto_df.iterrows():
    if "#markets" in row['link']:
        crypto_df["link"].iloc[index].replace("#markets", "")

print(crypto_df.shape)
        
for index,row in crypto_df.iterrows():
    link = row['link'] 
    name = link.split("/currencies/")[1][:-1]
    name=name.replace("-"," ")
    crypto_df.at[index, 'name'] = name
    crypto_df.at[index, 'link'] = f"https://coinmarketcap.com/{link}"
    crypto_df['link'] = crypto_df['link'].str.replace('//currencies/', '/currencies/')

(868, 2)
(868, 2)


In [84]:
print(crypto_df.sample(10))

                      name                                               link
726            locus chain  https://coinmarketcap.com/currencies/locus-chain/
632            peipei coin  https://coinmarketcap.com/currencies/peipei-coin/
2                  stellar      https://coinmarketcap.com/currencies/stellar/
269                 pixels       https://coinmarketcap.com/currencies/pixels/
384               terrausd     https://coinmarketcap.com/currencies/terrausd/
541                  myria        https://coinmarketcap.com/currencies/myria/
207  act i the ai prophecy  https://coinmarketcap.com/currencies/act-i-the...
830           og fan token  https://coinmarketcap.com/currencies/og-fan-to...
246           apu apustaja  https://coinmarketcap.com/currencies/apu-apust...
17                    aave         https://coinmarketcap.com/currencies/aave/


In [85]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

def scrape_network_links(url):
    # Initialize a UserAgent object
    user_agent = UserAgent()

    # Set headers with a random user agent
    headers = {'User-Agent': user_agent.random}

    # Send a GET request to the URL
    response = requests.get(url, headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract links based on class
        elements = soup.find_all('div', class_='ssc-65e7f566-0 clXVKn InfoBarItem_base__tan4P')

        # Extract links from each element
        links = []
        for element in elements:
            a_tags = element.find_all('a', {'target': '_blank'})
            links.extend([a['href'] for a in a_tags])

        # Extract text between specific markers
        page_source = str(soup)

        explorer_start = '"explorer":['
        explorer_end = '],"message_board":['

        start_index = page_source.find(explorer_start) + len(explorer_start)
        end_index = page_source.find(explorer_end)

        explorer_text = page_source[start_index:end_index]

        # Split the explorer_text into a list of links
        explorer_links = explorer_text.split(',')

        # Create a dictionary to store cleaned links with network names
        cleaned_links_dict = {}

        # Extract the cleaned explorer links
        for i, explorer_link in enumerate(explorer_links):
            cleaned_link = explorer_link.strip().strip('[]"')

            # Exclude links starting with "https://github.com/"
            if i == len(explorer_links) - 1:
                cleaned_link = cleaned_link.replace('source_code":["', '')
            
            # Exclude links starting with "https://github.com/"
            if not cleaned_link.startswith("https://github.com/"):
                # Extract network name from the link
                network_name = cleaned_link.split('//')[-1].split('.')[0]
                cleaned_links_dict[network_name] = cleaned_link

        return cleaned_links_dict

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return {}
    
    
network_list=[]
i=0
for index, row in crypto_df.iterrows():
    url = row['link']
    result = scrape_network_links(url)
    network_list=network_list+[result]
    print(result)

{'tonscan': 'https://tonscan.org/', 'ton': 'https://ton.app/explorers', 'tonviewer': 'https://tonviewer.com/', 'etherscan': 'https://etherscan.io/token/0x582d872a1b094fc48f5de31d3b73f2d9be47def1', 'www': 'https://www.okx.com/web3/explorer/eth/token/0x582d872a1b094fc48f5de31d3b73f2d9be47def1'}
{'etherscan': 'https://etherscan.io/token/0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce', 'solscan': 'https://solscan.io/token/CiKu4eHsVrc1eueVQeHn7qhXTcVu95gSQmBpX4utjL9z', 'bscscan': 'https://bscscan.com/token/0xb1547683DA678f2e1F003A780143EC10Af8a832B', 'finder': 'https://finder.terra.money/mainnet/address/terra1huku2lecfjhq9d00k5a8dh73gw7dwe6vvuf2dd', 'www': 'https://www.okx.com/web3/explorer/eth/token/0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce', 'source_code":': 'source_code":'}
{'dashboard': 'https://dashboard.stellar.org/', 'stellar': 'https://stellar.expert/explorer/public/', 'steexp': 'https://steexp.com/', 'blockchair': 'https://blockchair.com/stellar'}
{'polkadot': 'https://polkadot.js.org

In [86]:
print(network_list)

[{'tonscan': 'https://tonscan.org/', 'ton': 'https://ton.app/explorers', 'tonviewer': 'https://tonviewer.com/', 'etherscan': 'https://etherscan.io/token/0x582d872a1b094fc48f5de31d3b73f2d9be47def1', 'www': 'https://www.okx.com/web3/explorer/eth/token/0x582d872a1b094fc48f5de31d3b73f2d9be47def1'}, {'etherscan': 'https://etherscan.io/token/0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce', 'solscan': 'https://solscan.io/token/CiKu4eHsVrc1eueVQeHn7qhXTcVu95gSQmBpX4utjL9z', 'bscscan': 'https://bscscan.com/token/0xb1547683DA678f2e1F003A780143EC10Af8a832B', 'finder': 'https://finder.terra.money/mainnet/address/terra1huku2lecfjhq9d00k5a8dh73gw7dwe6vvuf2dd', 'www': 'https://www.okx.com/web3/explorer/eth/token/0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce', 'source_code":': 'source_code":'}, {'dashboard': 'https://dashboard.stellar.org/', 'stellar': 'https://stellar.expert/explorer/public/', 'steexp': 'https://steexp.com/', 'blockchair': 'https://blockchair.com/stellar'}, {'polkadot': 'https://polkadot.js

In [87]:
# Create the DataFrame where each row has a dictionary
network_list_df = pd.DataFrame({'network': network_list})

# Print the DataFrame
print(network_list_df)

                                               network
0    {'tonscan': 'https://tonscan.org/', 'ton': 'ht...
1    {'etherscan': 'https://etherscan.io/token/0x95...
2    {'dashboard': 'https://dashboard.stellar.org/'...
3    {'polkadot': 'https://polkadot.js.org/apps/#/e...
4    {'app': 'https://app.metrika.co/hedera/dashboa...
..                                                 ...
863                                                 {}
864                                                 {}
865                                                 {}
866                                                 {}
867                                                 {}

[868 rows x 1 columns]


In [88]:
crypto_df['network'] = network_list
print(crypto_df.sample(5))

             name                                               link  \
540        staika       https://coinmarketcap.com/currencies/staika/   
302          saga         https://coinmarketcap.com/currencies/saga/   
464  alien worlds  https://coinmarketcap.com/currencies/alien-wor...   
179      griffain     https://coinmarketcap.com/currencies/griffain/   
646     contentos    https://coinmarketcap.com/currencies/contentos/   

                                               network  
540  {'solscan': 'https://solscan.io/token/8BMzMi2X...  
302                                           {'': ''}  
464  {'wax': 'https://wax.bloks.io/tokens/TLM-wax-a...  
179  {'solscan': 'https://solscan.io/token/KENJSUYL...  
646                                                 {}  


In [91]:
import os

# Define the path for the directory and CSV file
directory = 'data'
file_path = os.path.join(directory, 'crypto_df.csv')

# Check if the directory exists; if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)

# Check if the file already exists and delete it
if os.path.exists(file_path):
    os.remove(file_path)

# Save the dataframe to the CSV file
crypto_df.to_csv(file_path, index=False)